# Web Scraping and APIs - Requests, Selenium and BeautifulSoup

This notebook provides a quick introduction to web scraping with Python, as part of the University of Amsterdam course Computational Social Science Analysis.

We will start by a simple introduction to using APIs and web scraping. We will learn scraping using requests and selenium. We will then build up toward a small research project that uses scraping and API.

It has been developed by Petter Törnberg. p.tornberg@uva.nl
Version 1.0. 2024-04-04 


# Interacting with APIs 

An API, or Application Programming Interface, allows different software applications to talk to each other, sharing data and functionalities easily. Developers use APIs to access features or data from other services, enabling more complex and feature-rich applications. Essentially, APIs serve as bridges between different software, making it possible for them to interact and share resources.



We're going to start with getting data from a simple API. It's easy!

## 1. Using a simple API:  How's the weather?

To fetch data from any API or website, we can use the requests package. The requests package abstracts the complexities of making requests behind simple API methods, allowing developers to send HTTP/1.1 requests with various methods like GET, POST, PUT, and others

In [ ]:
!pip install requests

As an example, we will use OpenWeatherMap.

#### API Documentation: _Read The Fine Manual! (RTFM)_
Public APIs always come with documentation that describes how to use the API, and what data you can expect. 

To find the OpenWeatherMap API, you can go to:
https://openweathermap.org/api


#### Getting the current weather
We will here use the current weather function, to get the current weather in Amsterdam.

In [1]:
import requests

api_key = "de26752686c975de6a1c38a998f50fec"
city_name = "Amsterdam"
base_url = "http://api.openweathermap.org/data/2.5/weather?"

# Complete URL for the API call
url = f"{base_url}q={city_name}&appid={api_key}"

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Here is the result from the API:")
    print(response.text)
    json_string = response.text
else:
    print("Error: Unable to get data from OpenWeatherMap API! :(")

Here is the result from the API:
{"coord":{"lon":4.8897,"lat":52.374},"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03n"}],"base":"stations","main":{"temp":285.35,"feels_like":284.74,"temp_min":284.18,"temp_max":285.97,"pressure":1005,"humidity":81},"visibility":10000,"wind":{"speed":7.2,"deg":240},"clouds":{"all":40},"dt":1712256371,"sys":{"type":2,"id":2012552,"country":"NL","sunrise":1712207222,"sunset":1712254754},"timezone":7200,"id":2759794,"name":"Amsterdam","cod":200}


#### Huh, what is this strange text?
As you can see, the result we get is in a particular text format. This format is called JSON (pronounced "Jason"), which is used by most APIs - both internal and public.

JSON (JavaScript Object Notation) is a data interchange format that is easy for humans to read and write and easy for machines to parse and generate. It is primarily used to transmit data between a server and a web application, serving as an alternative to XML, and is widely used for representing structured data and exchanging information in web development.


### Parsing JSON

Luckily, JSON is very easy to parse using Python. We may for instance turn it into a dict. We use the json library to do so.

In [2]:
import json

data = json.loads(json_string)

# Now parsed_data is a Python dictionary containing the data from the JSON string
main = data['main']
weather = data['weather']
print(f"{city_name:-^30}")
print(f"Temperature: {main['temp']}K")
print(f"Humidity: {main['humidity']}%")
print(f"Weather: {weather[0]['main']}")
print(f"Description: {weather[0]['description']}")

----------Amsterdam-----------
Temperature: 285.35K
Humidity: 81%
Weather: Clouds
Description: scattered clouds


#### Exercise 1: Your turn! Get the forecast!

Now your task is to get the "5 day / 3 hour forecast data" from the API, to figure out how the weather in Amsterdam will be in the coming days. Read the manual!

The goal is to print the date in the following format: 
- On 2023-10-06 12:00:00 the temperature will be 15 C
- On 2023-10-06 15:00:00 the temperature will be  4 C

etc.

There are two extra challenges here. 
First, the datetime is a timestamp (a float value representing the number of seconds since January 1, 1970, the Unix epoch), which you will need to convert to a readable date.

Second, you will need to convert the temperature from Kelvin to Celsius.

In [15]:
#Some help: a function to convert timestamp to date-time string
from datetime import datetime

def parse_timestamp(dt):
    dt_object = datetime.utcfromtimestamp(dt)
    formatted_date = dt_object.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_date

In [16]:
# SOLUTION:
import requests

api_key = "de26752686c975de6a1c38a998f50fec"
city_name = "Amsterdam"
base_url = "http://api.openweathermap.org/data/2.5/forecast?"

# Complete URL for the API call
url = f"{base_url}q={city_name}&appid={api_key}"

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    j = response.json()
    for l in j['list']:
        print(f"{parse_timestamp(l['dt'])}: {int(l['main']['temp']-273)}C")
else:
    print("Error: Unable to get data from OpenWeatherMap API! :(")

2024-04-05 09:00:00: 13C
2024-04-05 12:00:00: 13C
2024-04-05 15:00:00: 14C
2024-04-05 18:00:00: 13C
2024-04-05 21:00:00: 13C
2024-04-06 00:00:00: 13C
2024-04-06 03:00:00: 12C
2024-04-06 06:00:00: 12C
2024-04-06 09:00:00: 17C
2024-04-06 12:00:00: 22C
2024-04-06 15:00:00: 22C
2024-04-06 18:00:00: 18C
2024-04-06 21:00:00: 15C
2024-04-07 00:00:00: 15C
2024-04-07 03:00:00: 12C
2024-04-07 06:00:00: 12C
2024-04-07 09:00:00: 14C
2024-04-07 12:00:00: 17C
2024-04-07 15:00:00: 17C
2024-04-07 18:00:00: 14C
2024-04-07 21:00:00: 12C
2024-04-08 00:00:00: 11C
2024-04-08 03:00:00: 10C
2024-04-08 06:00:00: 10C
2024-04-08 09:00:00: 12C
2024-04-08 12:00:00: 11C
2024-04-08 15:00:00: 13C
2024-04-08 18:00:00: 13C
2024-04-08 21:00:00: 12C
2024-04-09 00:00:00: 9C
2024-04-09 03:00:00: 8C
2024-04-09 06:00:00: 8C
2024-04-09 09:00:00: 9C
2024-04-09 12:00:00: 11C
2024-04-09 15:00:00: 11C
2024-04-09 18:00:00: 8C
2024-04-09 21:00:00: 7C
2024-04-10 00:00:00: 7C
2024-04-10 03:00:00: 7C
2024-04-10 06:00:00: 7C


Now you have a sense of how to get data from a simple API!

## 2. Simple webscraping

Let's start by using _requests_ on a normal website instead. It's quite similar! We here use it to fetch the CSS programme website. 

In [1]:
import requests

url = "https://www.uva.nl/en/programmes/bachelors/computational-social-science/computational-social-science.html"

response = requests.get(url)

if response.status_code == 200:
    print("Here is the result:")
    print(f"{response.text[:300]} ...")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Here is the result:




<!doctype html>
<html class="no-js" lang="en">
<head>
    <meta charset="utf-8"/>

    <title>Bachelor's Computational Social Science - University of Amsterdam</title>
            <link rel="canonical" href="https://www.uva.nl/en/programmes/bachelors/computational-social-science/computational-so ...


As you can see, the result is in HTML: the simple markup language that the internet is built on.

To get data from HTML, we therefore need to parse the HTML to fetch the data that we are interested in. This is core to all scraping.

We therefore need a way of parsing the HTML to get the data that we are interested in.

This is where BeautifulSoup comes in!

### Requests + BeautifulSoup: Parsing HTML and XML

Beautiful Soup is a Python library designed for web scraping, to extract data from HTML and XML files. It provides ways for navigating, searching, and modifying "parse trees", making it easy to extract necessary information from a web page. 

Here follows a list of some of the main functionalities and methods provided by Beautiful Soup:

1. Parsing
- BeautifulSoup(markup, parser): This is the constructor function that takes in the HTML or XML content as a string and a string representing the parser to be used (like 'html.parser', 'lxml', 'lxml-xml', or 'html5lib').

2. Navigating the Parse Tree
- .contents: Provides a list of a tag’s children.
- .children: Similar to .contents but returns an iterator over the tag’s children.
- .descendants: Iterates over all of a tag’s children, recursively.
- .parent and .parents: Navigate up the parse tree to find parent tags.

3. Searching the Tree
- .find_all(name, attrs, recursive, string, limit, **kwargs): Searches the tree for tags matching the specified filters, and returns a ResultSet of matching tags.
- .find(name, attrs, recursive, string, **kwargs): Returns the first tag found that matches the specified filters.
- .select(css_selector): Searches the tree for tags matching the given CSS selector(s).

4. Accessing Tag Attributes
- .name: Accesses the name of the tag.
- .attrs: Accesses a dictionary of the tag’s attributes.

5. Manipulating the Tree
- .append(): Appends a tag or string to a tag’s contents.
- .insert(): Inserts a tag or string within a tag’s contents at a specified position.
- .extract(): Extracts a tag from the tree, modifying the tree.
- .decompose(): Removes a tag from the tree and then completely destroys it along with its contents.

6. Get Text
- .get_text(separator, strip, types): Extracts all text from a document or tag, optionally separated by a separator string and optionally stripping whitespace.

7. Formatting Output
- .prettify(): Returns a string representing the tag’s contents, formatted as nicely indented HTML.

8. Encoding
- .encode(encoding, formatter): Returns a string or bytes representing the tag, encoded in the given encoding.

9. Decoding
- .decode(formatter): Returns a Unicode string representing the tag.
- Beautiful Soup provides flexible and user-friendly ways to navigate and search the parse tree, making it a powerful tool for web scraping needs.




Let's use it to fetch the description of the CSS program!

In [ ]:
# Install the library if you do not already have it
!pip install beautifulsoup4

In [11]:
#Load the library
from bs4 import BeautifulSoup

### Parsing a simple example website with beautifulsoup
As you may know, HTML is hierarchically structured  - sometimes referred to as an HTML parse tree or the DOM tree. The DOM is a tree data structure that represents the hierarchical structure of an HTML document. Each node in the tree corresponds to an element (or "tag") in the HTML document, and the edges represent the nesting relationships between the elements. The root of the tree is typically the <html> tag, and it has child nodes representing the head and body of the HTML document, and those child nodes, in turn, have their own child nodes representing nested elements within them.

For example, consider a simple HTML document:


In [22]:
simplehtml = '''<html>
    <head>
        <title>My Page</title>
    </head>
    <body>
        <h1>Welcome to My Page</h1>
        <p>This is a paragraph.</p>
    </body>
</html>'''

The HTML parse tree for this document would have the <html> tag as the root, with child nodes \<head> and \<body>. The \<head> node would have a child node \<title>, and the \<body> node would have child nodes \<h1> and \<p>. Each node may also contain text nodes representing the text content within the tags.

Creating a parse tree from an HTML document is part of web scraping to navigate and extract information from the document structure programmatically. This is where BeautifulSoup comes in.


In [30]:
soup = BeautifulSoup(simplehtml, 'html.parser')

# Navigating to the <head> tag
head = soup.head
print("Head Tag:", head)

# Navigating to the <title> tag inside <head>
title = head.title
print("Title Tag:", title)

# Extracting text from the <title> tag
title_text = title.get_text()
print("Title Text:", title_text)

# Navigating to the <body> tag
body = soup.body
print("Body Tag:", body)

# Navigating to the <h1> tag inside <body>
h1 = body.h1
print("H1 Tag:", h1)

# Extracting text from the <h1> tag
h1_text = h1.get_text()
print("H1 Text:", h1_text)

# Navigating to the <p> tag inside <body>
paragraph = body.p
print("Paragraph Tag:", paragraph)

# Extracting text from the <p> tag
paragraph_text = paragraph.get_text()
print("Paragraph Text:", paragraph_text)

# We can navigate upward in tree with parent
print("Parent of h1:", h1.parent.name)

Head Tag: <head>
<title>My Page</title>
</head>
Title Tag: <title>My Page</title>
Title Text: My Page
Body Tag: <body>
<h1>Welcome to My Page</h1>
<p>This is a paragraph.</p>
</body>
H1 Tag: <h1>Welcome to My Page</h1>
H1 Text: Welcome to My Page
Paragraph Tag: <p>This is a paragraph.</p>
Paragraph Text: This is a paragraph.
Parent of h1: body


### Exercise: Parse a simple website
Your task is to parse the following simple website using beautifulsoup, and extract a dataframe that has the products listed, with their name, description, and price in separate columns.


In [17]:
from IPython.display import display, HTML
website_html = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Simple Website Example</title>
</head>
<body>
<h1>Welcome to Our Simple Website</h1>
<p>This is a demonstration of a simple HTML website designed for parsing practice.</p>
<h2>About Us</h2>
<p>We are a team dedicated to learning web scraping with BeautifulSoup.</p>
<h3>Contact Information</h3>
<p>Email us at: <a href="mailto:info@example.com">info@example.com</a></p>
<h2>Our Products</h2>
<table border="1">
    <tr>
        <th>Product Name</th>
        <th>Description</th>
        <th>Price</th>
    </tr>
    <tr>
        <td>Product 1</td>
        <td>An essential item for beginners.</td>
        <td>$19.99</td>
    </tr>
    <tr>
        <td>Product 2</td>
        <td>A must-have for advanced users.</td>
        <td>$29.99</td>
    </tr>
    <tr>
        <td>Product 3</td>
        <td>Now with bacon-flavor!</td>
        <td>$39.99</td>
    </tr>
</table>

</body>
</html>'''
print("This is how the website looks:")
display(HTML(website_html))

This is how the website looks:


Product Name,Description,Price
Product 1,An essential item for beginners.,$19.99
Product 2,A must-have for advanced users.,$29.99
Product 3,Now with bacon-flavor!,$39.99


In [14]:
# Import pandas
import pandas as pd 

# Parse the HTML
soup = BeautifulSoup(website_html, 'html.parser')

# [YOUR SOLUTION HERE]
#----- SOLUTION -----
# Find the table containing products
product_table = soup.find('table')

# Extract the rows in the table, skipping the header row
rows = product_table.find_all('tr')[1:]

# Extract the data for each row
products = []
for row in rows:
    cols = row.find_all('td')
    product_name = cols[0].text
    description = cols[1].text
    price = cols[2].text
    products.append([product_name, description, price])

# Convert the data into a DataFrame
df = pd.DataFrame(products, columns=['Product Name', 'Description', 'Price'])
#------ /SOLUTION -----

# Dataframe that has the products
df

,Product Name,Description,Price
0,Product 1,An essential item for beginners.,$19.99
1,Product 2,A must-have for advanced users.,$29.99
2,Product 3,Now with bacon-flavor!,$39.99


## A realistic example: Parsing the CSSci website
Let's use this to parse the CSSci website we fetched earlier.

In [4]:
#Parse the html
soup = BeautifulSoup(response.text, 'html.parser')


So, how do we find the element we want, in a complex HTML website like UvA.nl?

One way is to find out the CSS selector for the element you are seeking, you can use the extremely useful Chrome Developer Tools. Open Chrome. Go to the website. Go to Menu > More Tools > Developer Tools.

You can use the "Select element", represented by a diagonal arrow in the upper right corner. 

Click the element on the page that you are interested in: the main description.

You can now see that the description text is inside a _p_ of class _lead_ which is inside a _div_ with class 'c-programmepageheader'.

We can use the CSS selector to select this element:

In [12]:
description = soup.select('div.c-programmepageheader p.lead')

The result is a list with a single element in it: the element we are looking for. To get the text of this element, we simply need to:

In [19]:
print(description[0].get_text())

Would you like to learn how to make the world a better place by using digital technology? Are you eager to learn all about data science? Do you have a hands on attitude and excellent team spirit? Digitisation poses new questions and challenges for our societies, but it also brings new opportunities to intervene in societal issues such as social inequality and climate change. Scientifically grounded in social sciences, humanities and information science, as a student of Computational Social Science you will develop critical skills to analyse digital phenomena using advanced data science techniques combined with rigorous research methods. You will also learn how to design digital interventions and effect change to benefit society.


### Exercise: "Is Computational Social Science right for you?"
Your task is to find the list of four points on the UvA CSSci website that answers the questions of whether the CSS programme is right for you. 

Use what you've learned to fetch this list, and print each point.


In [1]:
# [YOUR CODE HERE]
# --- SOLUTION ---
points = soup.select('div.c-richtext__rowcontainer div div ul li')
print(f"Here are the {len(points)} points to decide if CSS is right for you:")
for i,point in enumerate(points):
    print(f"{i}. {point.text}")
# --- /SOLUTION ---

## Selenium: Scraping dynamic pages

While _requests_ is a powerful tool for getting static HTML pages, most websites these days are not static HTML. _requests_ cannot handle Javascript pages or dynamic content.

This is where Selenium comes in. Selenium automates a web browser, allowing it to interact with the JavaScript and dynamically loaded content on the webpage, thereby providing access to content modified or loaded by JavaScript after the initial page load. Selenium can also automate interactions with the website, such as clicking buttons, filling out forms, or navigating through pages. For these points, the requests library alone would be insufficient, as it cannot interact with webpage elements or execute user-like actions.

Selenium in other words runs a complete web browser, and automates clicking on the websites. This allows it to scrape nearly any website. But it also means that it is relatively heavy and slow, compared to a _requests_ based solution. 

*Takeaway: use requests when dealing with static websites or APIs. Use Selenium when dealing with more complex dynamic websites.*


### Installing Selenium

Installing selenium can be a bit of a challenge on its own, as it is dependent on having a chrome/chromium browser installed. Expect a bit of fiddling!

Regardless of the OS, you first need to install the Selenium python package: 





In [ ]:
!pip install selenium

!pip install webdriver-manager


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")

In [42]:
#You should now see a web browser opening on your computer 

In [14]:
#Let's first close the cookie window by click the "no thanks" button
button = driver.find_element('id','W0wltc')
button.click()

In [15]:
# Find the search bar using the name of the input field
search_bar = driver.find_element("name", "q")

# Type the search term and hit ENTER
search_bar.send_keys("university of amsterdam")

In [18]:
#Click enter!
search_bar.send_keys(Keys.RETURN)

# Wait for some time to let the results load
time.sleep(2) 

#The page will now have made the search!

In [100]:
# Locate the titles and URLs of the search hits.
results = driver.find_elements(by=By.CSS_SELECTOR,value='a h3') #Select all links under the div with id search: these are the search results.

# Extract and print the top 10 hits 
for result in results:
    if len(result.text)>0:
        #This is how to get the parent element in selenium. We want the <a> to get the URL.
        parent_element = result.find_element(by=By.XPATH, value='..')
        
        print(f"{result.text}. {parent_element.get_attribute('href')}")

-----
To get more results, we can scroll to the bottom of the page, and wait for a moment

In [19]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Wait for a while to allow contents to load, if any
time.sleep(2)


In [22]:
# Running the same code as above, we now get a larger number of results!
results = driver.find_elements(by=By.CSS_SELECTOR,value='a h3') #Select all links under the div with id search: these are the search results.

# Extract and print the top 10 hits 
i=0
for result in results:
    if len(result.text)>0:
        i+=1
        #This is how to get the parent element in selenium. We want the <a> to get the URL.
        parent_element = result.find_element(by=By.XPATH, value='..')
        
        print(f"{i}. {result.text}. {parent_element.get_attribute('href')}")

In [21]:
# Close the browser window
driver.quit()

### Exercise: Find the Google ranking of UvA's Computational Social Science. 

Your task is to adapt the code to use selenium to search for 'computational social science', and to find where UvA shows up in the search ranking. 

1. Use Selenium to open google.com. Close the popup, and search for 'computational social science'

2. Your script should keep scrolling in the search result until it finds a search result with an HREF that includes with 'uva.nl'. 

3. It should then print the number in the list of the identified link, and how many pages you had to scroll. For instance, if it is the first link found, your code should output: 'UvA was number 1 link for search result, on page 1 in the Google ranking!'


In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# This function fetches
# Takes: a search term string
# Returns: the link in order it was found on, and the page it was found on. e.g., "return rank, page"
# If it does not find the link in the first 5 pages, "return None, None"
def find_google_ranking(search_term,url_to_look_for):
    #[YOUR CODE HERE]

rank, page = find_google_ranking("computational social science","uva.nl")

if rank is None:
    print(f"Uva.nl was not listed in the first {how_many_pages_to_try} pages! :( We need to work on our SEO!")
else:
    print(f"UvA was number {rank} link for search result, on page {page} in the Google ranking!")
    

In [ ]:
# SOLUTION
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# This function fetches
# Takes: a search term string
# Returns: the link in order it was found on, and the page it was found on.
def find_google_ranking(search_term,url_to_look_for):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://www.google.com")

    time.sleep(5) 

    #Let's first close the cookie window by click the "no thanks" button
    button = driver.find_element('id','W0wltc')
    button.click()

    time.sleep(3) 

    # Find the search bar using the name of the input field
    search_bar = driver.find_element("name", "q")

    # Type the search term and hit ENTER
    search_bar.send_keys(search_term)

    #Click enter!
    search_bar.send_keys(Keys.RETURN)

    # Wait for some time to let the results load
    time.sleep(2) 

    how_many_pages_to_try=5
    #The page will now have made the search
    # We go over pages by scrolling down
    for page in range(how_many_pages_to_try):

        # Get search results
        results = driver.find_elements(by=By.CSS_SELECTOR,value='a h3') 

        #Extract hits
        for result_nr,result in enumerate(results):
            if len(result.text)>0:

                parent_element = result.find_element(by=By.XPATH, value='..')

                #Does it contain the URL to uva?
                if parent_element.get_attribute('href') is not None and url_to_look_for in parent_element.get_attribute('href'):
                    #We found it!
                    driver.quit()
                    return result_nr+1, page+1

        #Scroll to next page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) # Wait for a while to allow contents to load

    driver.quit()
    return None,None

rank, page = find_google_ranking("computational social science","uva.nl")

if rank is None:
    print(f"Uva.nl was not listed in the first {how_many_pages_to_try} pages! :( We need to work on our SEO!")
else:
    print(f"UvA was number {rank} link for search result, on page {page} in the Google ranking!")
    

## More advanced API: How toxic are YouTube comments? Combining YouTube API and Perspective API
In this part of the guide, we will use YouTube API to collect comments from videos. 

### About authentication
APIs often require users to sign up and use credentials. These are often based on "API keys" which link a call to the API to a particular user or registered application. There are many reason for APIs requiring authentifiatoin: by requiring credentials, API providers can control access to the data or services they offer, preventing unauthorized access and abuse, and to ensure rate limiting - that is, managing the load on the server by restricting the number of API calls from a single user or application within a given time frame.


You can sign up to the YouTube API at https://developers.google.com/youtube/v3 
Read about the process on: https://developers.google.com/youtube/v3/getting-started

Google offers a range of powerful and interesting APIs, both for data collection and analysis. Have a look and browse their offerings.



### Fetching YouTube comments
We will now use the YouTube API to fetch comments associated to a particular YouTube video.

You'll find the API documentation here: https://developers.google.com/youtube/v3/docs/commentThreads 


In [ ]:
import requests
import time

# The API key is your key to the YouTube API. You will neeed to get your own. To do so, visit https://developers.google.com/youtube/v3/getting-started 
api_key = #[YOUR API KEY]
video_id = "dQw4w9WgXcQ"  
# Replace with the ID of the video you are interested in. 
# You can find the ID by going to a video in Youtube, and getting the string after v= in the URL. For instance, i0EfLMe5FGk in https://www.youtube.com/watch?v=i0EfLMe5FGk

url = f"https://www.googleapis.com/youtube/v3/commentThreads"
params = {
    'part': 'snippet',
    'videoId': video_id,
    'maxResults': 100,  # max number of comments to fetch 
    'textFormat': 'plainText',
    'key': api_key,
}

all_comments = []

maximum_pages = 3 #How many pages to get at most

for page in range(maximum_pages):
    print(f"Getting page {page}...")
    response = requests.get(url, params=params)
    if response.status_code == 200:
        result_json = response.json()
        all_comments.extend([item['snippet']['topLevelComment']['snippet']['textDisplay'] for item in result_json.get('items', [])])

        # Many APIs provide the result page by page. If there is another page, this API returns a nextPageToken, that we can
        # send to the API to get the next page in line. If there are no more comments, there will be no such token.
        if 'nextPageToken' in result_json:
            params['pageToken'] = result_json['nextPageToken']
            
            # Ensure you don't hit the quota limits by adding a delay
            time.sleep(1)
        else: #No token, so no more pages
            break
    else:
        print("Error: ", response.status_code)
        break

# Now 'all_comments' list contains all the comments from the video
print(f"Done. Fetched {len(all_comments)} comments!")


Getting page 0...
Getting page 1...
Getting page 2...
Getting page 3...
Getting page 4...
Done! Fetched 490 comments!


In [ ]:
#Print the first five comments
print(all_comments[:5])

['1 BILLION views for Never Gonna Give You Up!\xa0 Amazing, crazy, wonderful! Rick ♥️', "Greetings from Japan. This melody gives me a strong sense of déjà vu. It feels like something is in the right place. It's a very good song. I definitely love this song and find it nostalgic.", '13 years still watching this masterpiece \U0001fae1\U0001fae1', 'Chính nó:)))', 'I love this song\n\n\n\n\n\nEdit:CAN I GET 178 LIKES?🗣️🔥🔥']


### Perspective Toxicity API

The Perspective API, developed by Jigsaw and Google's Counter Abuse Technology team, is a tool that leverages machine learning to score toxicity in online conversation. The API provides various models to assess different aspects of conversations, like toxicity, severe toxicity, and threat, allowing developers and service providers to automatically moderate content that is harmful, abusive, or likely to drive users away, thus fostering healthier and more respectful online interactions.

Perspective API is an example of an API that can be used to analyze your own data, rather than just fetching existing data.



Perspective API is easiest to use through the Python package offered by Google. Many APIs offer Python packages to make it easier to use the API. APIs offer packages to simplify and streamline the interaction between the end-user's code and the API’s endpoints, abstracting the intricacies of HTTP requests, response handling, and error handling.


In [ ]:
#Install the package
!pip install googleapiclient

In [ ]:
from googleapiclient import discovery

In [ ]:
PERSPECTIVE_API_KEY = # YOUR API KEY HERE

# The text string you want to analyze
message = "I fart in your general direction! Your mother was a hamster and your father smelt of elderberries!"

client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=PERSPECTIVE_API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

analyze_request = {
  'comment': { 'text': message },
  'requestedAttributes': {'TOXICITY': {}}
}

# Don't overload the API
time.sleep(0.5)

response = client.comments().analyze(body=analyze_request).execute()

toxicity = response['attributeScores']['TOXICITY']['summaryScore']['value']

print(f"The message scores {toxicity} in toxicity")


The message scores 0.85850734 in toxicity


### Mini-project: How toxic are the YouTube comments?
Your task is to write a script that: 

1. Takes a list of YouTube video IDs and collects the first 100 comments from each video.
2. Calculate the toxicity of each comment on the videos using Perspective API, and stores the result in a pandas Dataframe.
3. Shows how toxic the comments are on average according to the Perspective API. (Use for instance np.mean() to calculate the average toxicity.)

Select a couple of Youtube videos of your own choice, and use your code to analyze which of them has the most toxic comments. Reflect about the meaning of the findings.


In [ ]:
# SOLUTION

import requests
import time
from googleapiclient import discovery
import pandas as pd

youtube_api_key = [YOUR KEY HERE]
PERSPECTIVE_API_KEY = [YOUR KEY HERE]

client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=PERSPECTIVE_API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

#This function returns a list of comments (strings) associated to a video on Youtube
def fetch_comments_for_video(video_id, max_comments_to_fetch=100):

    print(f"Fetching comments for video {video_id}.")
    
    url = f"https://www.googleapis.com/youtube/v3/commentThreads"
    params = {
        'part': 'snippet',
        'videoId': video_id,
        'maxResults': max_comments_to_fetch,  
        'textFormat': 'plainText',
        'key': youtube_api_key,
    }

    all_comments = []
    page = 0
    while(True):
        page+=1
        
        if len(all_comments)>=max_comments_to_fetch:
            break
        
        print(f"Getting page {page}...")
        response = requests.get(url, params=params)
        if response.status_code == 200:
            result_json = response.json()
            all_comments.extend([item['snippet']['topLevelComment']['snippet']['textDisplay'] for item in result_json.get('items', [])])

            # Many APIs provide the result page by page. If there is another page, this API returns a nextPageToken, that we can
            # send to the API to get the next page in line. If there are no more comments, there will be no such token.
            if 'nextPageToken' in result_json:
                params['pageToken'] = result_json['nextPageToken']

                # Ensure you don't hit the quota limits by adding a delay
                time.sleep(1)
            else: #No token mean no more pages, so we're done
                break
        else:
            print("Error: ", response.status_code)
            break

    # Now 'all_comments' list contains all the comments from the video
    print(f"Done. Fetched {len(all_comments)} comments!")
    return all_comments

#This function uses fetch_comments_for_video() to collect comments for several videos. 
# It takes a list of video ids and returns a dataframe  with the structure:
# video_id | comment
# 
def fetch_comments_for_videos(list_of_video_ids):
    l = []
    for video_id in list_of_video_ids:
        comments = fetch_comments_for_video(video_id,)
        for comment in comments:
            l.append({'video_id':video_id,'comment': comment})
                 
    return pd.DataFrame(l)

# This measures the toxicity of a single message using the Perspective API
def measure_toxicity_of_message(message):
    
    analyze_request = {
      'comment': { 'text': message },
      'requestedAttributes': {'TOXICITY': {}}
    }

    time.sleep(0.1)

    response = client.comments().analyze(body=analyze_request).execute()

    toxicity = response['attributeScores']['TOXICITY']['summaryScore']['value']

    return toxicity


In [ ]:
#Trump's state of the union vs Biden's state of the union
comments = fetch_comments_for_videos(['ATFwMO9CebA','Wl6b5KnpmB4'])

Fetching comments for video ATFwMO9CebA.
Getting page 1...
Getting page 2...
Done. Fetched 199 comments!
Fetching comments for video Wl6b5KnpmB4.
Getting page 1...
Done. Fetched 100 comments!


In [ ]:
#Prepare the dataframe
comments['toxicity'] = None
comments['analyzed'] = False    

In [ ]:
#This is a simple way of structuring your code when scraping many pages.
i = 0
nrfailed = 0
while(True):    
    #Fetch a random row
    left_to_process = comments.loc[comments['analyzed']==False]
    
    if len(left_to_process)==0:
        print(f"We're done! Analysis failed for {nrfailed} of {len(comments)}.")
        break
    
    else:
        comment = left_to_process.sample(1)
        index = comment.index[0]
        message = comment.comment.values[0]

        #Keep track of progress. Every 10 measures, we print out a progress report
        i+=1
        if i%10==0:
            print(f"{len(comments.loc[comments['analyzed']==False])} comments left out of {len(comments)}...")

        try:
            #Analyze toxicity
            toxicity = measure_toxicity_of_message(message)
            comments.loc[index,'toxicity'] = toxicity

        except Exception as e:
            #The API will fail for mant comments, for instance if they are too short or in the wrong language.
            nrfailed+=1
        finally:
            comments.loc[index,'analyzed'] = True

290 comments left out of 299...
280 comments left out of 299...
270 comments left out of 299...
260 comments left out of 299...
250 comments left out of 299...
240 comments left out of 299...
230 comments left out of 299...
220 comments left out of 299...
210 comments left out of 299...
200 comments left out of 299...
190 comments left out of 299...
180 comments left out of 299...
170 comments left out of 299...
160 comments left out of 299...
150 comments left out of 299...
140 comments left out of 299...
130 comments left out of 299...
120 comments left out of 299...
110 comments left out of 299...
100 comments left out of 299...
90 comments left out of 299...
80 comments left out of 299...
70 comments left out of 299...
60 comments left out of 299...
50 comments left out of 299...
40 comments left out of 299...
30 comments left out of 299...
20 comments left out of 299...
10 comments left out of 299...
We're done! Analysis failed for 18 of 299.


In [ ]:
comments

,video_id,comment,toxicity,analyzed
0,ATFwMO9CebA,We love you president Trump!,0.015205,True
1,ATFwMO9CebA,Take a lesson people! That's what a real presi...,0.045131,True
2,ATFwMO9CebA,🕊️🏡🕊️💝💝💝💝💝💝💝💝💝💝💝💝💝💝💝💝💝💝🦅💝💝💝💝💝💝🌧️💝💝💝💝💝💝💝💝💝💝🦅🦅🦅🦅...,0.042657,True
3,ATFwMO9CebA,This is the real state of the union,0.028149,True
4,ATFwMO9CebA,How great. 2 people who ran for vice president...,0.05849,True
...,...,...,...,...
294,Wl6b5KnpmB4,ខ្ញុំchea den ខ្ញុំមិនទាន់បានលុយទេសូមលោកជូបៃឌិ...,None,True
295,Wl6b5KnpmB4,Refund your money if your airplanes delayed. W...,0.034277,True
296,Wl6b5KnpmB4,He slipped up 31:02 and was gonna say came tog...,0.071337,True
297,Wl6b5KnpmB4,Who’s the girl in the yellow dress she looks l...,0.157667,True


In [ ]:
#Let's compare the toxicities.
# We look at the mean toxicity for the successfully analyzed comments:
comments.loc[~comments['toxicity'].isna()].groupby(['video_id'])['toxicity'].mean()

video_id
ATFwMO9CebA    0.214887
Wl6b5KnpmB4    0.202312
Name: toxicity, dtype: float64

## Mini-project 2: Does the YouTube algorithm radicalize?

We will now go through a more complex exercise, for a small research paper.

Researchers have argued that the YouTube autoplay feature can lead to radicalization. The platform's recommendation system is designed to keep users engaged for as long as possible. The algorithm achieves this by suggesting content that it predicts the user will find interesting or compelling, based on their viewing history, search terms, and other interactions. 

However, critics argue that this approach can create a "filter bubble," where users are only exposed to content and perspectives similar to those they have already encountered, thereby reinforcing existing beliefs and opinions. There are concerns that this can lead to the incremental presentation of more extreme content, as users are gradually exposed to increasingly radical viewpoints in a bid to sustain engagement. This phenomenon, sometimes referred to as "algorithmic radicalization," has sparked debates about the ethical responsibilities of social media and content-sharing platforms and their role in the spread of misinformation, hate speech, and extremist ideologies. 

This would suggest that the comments on videos become more and more toxic as the algorithm proceeds!

In this exercise, we are going to explore this hypothesis by tracing the YouTube autoplay feature from a given starting point.

You will use the code we developed in our previous practical to fetch the comments on the video and evaluate their average toxicity. In case you did not complete that task, we will give you the solution here.

While the Youtube API gives access to some features, the "next video" feature is not accessible through the API. For this, we therefore need to scrape the interface.

#### Task: 

1. Choose a video to start from. This might for instance be a political video, where you would expect a radicalization loop to take place.

2. Write code to repeatedly go to the "next upcoming video" and store the number of steps taken, the video id and the title. (Remember to pause between each fetch, so the page has time to load.)

3. Store at least 10 steps of "next video", so that a trend can be spotted.

4. Use your code from the previous practical, where you collected comments on YouTube videos using the API, and calculated the toxicity of each comment.

5. For each video, calculate the average toxicity of the comments. 

6. Plot the trend: are the comments becoming more toxic? Do your findings fit with the YouTube autoplay radicalization hypothesis?


#### Code for analyzing how toxic text is
We will use the Perspective API to measure toxicity. It's a machine learning API that classifies how incivil a social media message is. 

Make sure that you go through this code and understand it!

In [68]:
import requests
import time
from googleapiclient import discovery
import pandas as pd

api_key = [YOUR PERSPECTIVE API KEY HERE]

client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=PERSPECTIVE_API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

#This function returns a list of comments (strings) associated to a video on Youtube
def fetch_comments_for_video(video_id, max_comments_to_fetch=100):

    print(f"Fetching comments for video {video_id}.")
    
    url = f"https://www.googleapis.com/youtube/v3/commentThreads"
    params = {
        'part': 'snippet',
        'videoId': video_id,
        'maxResults': 100,  
        'textFormat': 'plainText',
        'key': api_key,
    }

    all_comments = []
    page = 0
    while(True):
        page+=1
        
        if len(all_comments)>=max_comments_to_fetch:
            break
        
        print(f"Getting page {page}...")
        response = requests.get(url, params=params)
        if response.status_code == 200:
            result_json = response.json()
            all_comments.extend([item['snippet']['topLevelComment']['snippet']['textDisplay'] for item in result_json.get('items', [])])

            # Many APIs provide the result page by page. If there is another page, this API returns a nextPageToken, that we can
            # send to the API to get the next page in line. If there are no more comments, there will be no such token.
            if 'nextPageToken' in result_json:
                params['pageToken'] = result_json['nextPageToken']

                # Ensure you don't hit the quota limits by adding a delay
                time.sleep(1)
            else: #No token mean no more pages, so we're done
                break
        else:
            print("Error: ", response.status_code)
            break

    # Now 'all_comments' list contains all the comments from the video
    print(f"Done. Fetched {len(all_comments)} comments!")
    return all_comments

# This measures the toxicity of a single message using the Perspective API
def measure_toxicity_of_message(message):
    
    analyze_request = {
      'comment': { 'text': message },
      'requestedAttributes': {'TOXICITY': {}}
    }

    time.sleep(0.1)

    response = client.comments().analyze(body=analyze_request).execute()

    toxicity = response['attributeScores']['TOXICITY']['summaryScore']['value']

    return toxicity


# We first adapt our fetch_comments_for_videos() from last week so that it preserves our additional information about the video.
# This function now takes a list of dicts (created by the function above), and fetches the comments for each video.
# It produces a dataframe where each line is a comment, and the video information is included:
# [{'step':0,'video_id': 'ATFwMO9CebA', 'President Trump 2018 State of the Union Address (C-SPAN)', 'comment':'Great speech!' }...]
def fetch_comments_for_videos(list_of_videos,max_comments_to_fetch=100):
    list_of_comments = []
    for video in list_of_videos:
        comments = fetch_comments_for_video(video['video_id'],max_comments_to_fetch)
        for comment in comments:
            list_of_comments.append(video | {'comment': comment}) #Add comment information to video information
                 
    return pd.DataFrame(list_of_comments)

#This function takes a dataframe with comments, and analyzes each comment using perspective.
# It returns an updated dataframe with toxicity information for each comment.
def analyze_toxicity_of_comments(comments):
    #Prepare the dataframe
    comments['toxicity'] = None
    comments['analyzed'] = False    
    #This is a simple way of structuring your code when scraping many pages.
    i = 0
    nrfailed = 0
    while(True):    
        #Fetch a random row
        left_to_process = comments.loc[comments['analyzed']==False]

        if len(left_to_process)==0:
            print(f"We're done! Analysis failed for {nrfailed} of {len(comments)}.")
            break

        else:
            comment = left_to_process.sample(1)
            index = comment.index[0]
            message = comment.comment.values[0]

            #Keep track of progress. Every 10 measures, we print out a progress report
            i+=1
            if i%10==0:
                print(f"{len(comments.loc[comments['analyzed']==False])} comments left out of {len(comments)}...")

            try:
                #Analyze toxicity
                toxicity = measure_toxicity_of_message(message)
                comments.loc[index,'toxicity'] = toxicity

            except Exception as e:
                #The API will fail for mant comments, for instance if they are too short or in the wrong language.
                nrfailed+=1
            finally:
                comments.loc[index,'analyzed'] = True

    return comments

In [70]:
# In the previous exercise, we used this to compare the state of the union speeches of Trump and Biden:

#Fetch the comments: 
comments = fetch_comments_for_videos([{'president':'Trump','video_id':'ATFwMO9CebA'},{'president':'Biden','video_id':'Wl6b5KnpmB4'}],max_comments_to_fetch=50)

#Analyze comments:
comments = analyze_toxicity_of_comments(comments)

#Calculate average toxicity:
print("Average video comment toxicity:")
comments.loc[~comments['toxicity'].isna()].groupby(['president'])['toxicity'].mean()

# Who was more toxic? 

### Additional snippets of code to help you 

In [102]:
# Go to the initial video URL. You can modify this to your preferred video
driver.get('https://www.youtube.com/watch?v=dQw4w9WgXcQ')  

In [119]:
# Close cookie popup
buttons = driver.find_elements(by=By.CSS_SELECTOR,value='button')

for button in buttons:
    if 'Reject' in button.text:
        button.click()

In [133]:
#Get the title and id of the current video
video_id = driver.current_url.split('=')[1]
title = driver.find_element(by=By.CSS_SELECTOR,value='div#title h1')
print(f"Id: {video_id}. Title: {title.text}")

Id: PvHGl3L0WUI. Title: Karine Jean-Pierre has no answer for the crisis at the border.


In [142]:
# Click next video
nextvid = driver.find_element(by=By.CSS_SELECTOR,value='ytd-compact-video-renderer.ytd-watch-next-secondary-results-renderer a')
nextvid.click()

### TASK: Your code goes here

In [ ]:
# This function takes a video_id to start with, and then takes nr_steps of "next video" from that video.
# It returns a list of dicts, each containing with the step number, the video_id, and the title of the video
#  e.g., [{'step':0,'video_id': 'ATFwMO9CebA', 'President Trump 2018 State of the Union Address (C-SPAN)' }...]

def follow_next_video(start_video_id,nr_steps):
#[YOUR CODE HERE]

# This should result in the next 10 steps from the Biden state of the union video
list_of_videos = follow_next_video('Wl6b5KnpmB4',10)



In [67]:
list_of_videos
#Should look more or less like this:
# [{'step': 0,
#   'video_id': 'Wl6b5KnpmB4',
#   'title': 'President Joe Biden delivers 2023 State of the Union address to Congress — 2/7/23'},
#  {'step': 1,
#   'video_id': 'FtzvOZNyXdw',
#   'title': "Rise, fall of Sam Bankman-Fried, FTX at center of Michael Lewis' new book | 60 Minutes"},
#  {'step': 2,
#   'video_id': 'XqwGt69pDXQ',
#   'title': 'The Collapse Of FTX: Insiders Tell All | CNBC Documentary'},
#  {'step': 3,
#   'video_id': 'gqDCrdZVZnk',
#   'title': 'The world’s most dangerous arms dealer | DW Documentary'},
#  {'step': 4, 'video_id': 'G1p6rlDCxq0', 'title': 'World War One (ALL PARTS)'},
# ...

## What can you observe about the videos? 
# For instance, does the recommendation algorithm gets stuck in a cycle between two or three videos? 

In [64]:
#Now we're going to use our old code to analyze the data!

#Fetch comments for the videos
comments = fetch_comments_for_videos(list_of_videos,max_comments_to_fetch=50)

In [65]:
#Calculate toxicity of each comment. This will take a while!
comments = analyze_toxicity_of_comments(comments)

In [57]:
#Let's plot the average toxicity over time. Is there a clear trend?
comments.loc[~comments['toxicity'].isna()].groupby(['step'])['toxicity'].mean().plot()

In [ ]:
# SOLUTION:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd

# This function takes a video_id to start with, and then takes nr_steps of "next video" from that video.
# It returns a list of dicts, each containing with the step number, the video_id, and the title of the video
#  e.g., [{'step':0,'video_id': 'ATFwMO9CebA', 'President Trump 2018 State of the Union Address (C-SPAN)' }...]
def follow_next_video(start_video_id,nr_steps):
    l = []
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(f'https://www.youtube.com/watch?v={start_video_id}')  
    time.sleep(8)

    #Close popup
    buttons = driver.find_elements(by=By.CSS_SELECTOR,value='button')
    for button in buttons:
        if 'Reject' in button.text:
            button.click()
    
    time.sleep(2)
    
    #Get information for first video
    video_id = driver.current_url.split('=')[1]
    title = driver.find_element(by=By.CSS_SELECTOR,value='div#title h1')    
    l.append({'step':0, 'video_id': video_id, 'title': title.text})

    # Click next video repeatedly
    for step in range(nr_steps):
        # Click next video
        nextvid = driver.find_element(by=By.CSS_SELECTOR,value='ytd-compact-video-renderer.ytd-watch-next-secondary-results-renderer a')
        nextvid.click()
        time.sleep(5)
        
        #Fetch video_id and video title
        video_id = driver.current_url.split('=')[1]
        title = driver.find_element(by=By.CSS_SELECTOR,value='div#title h1')    
        l.append({'step':step+1, 'video_id': video_id, 'title': title.text})

    driver.quit()
    
    return l

list_of_videos = follow_next_video('Wl6b5KnpmB4',10)